<a href="https://colab.research.google.com/github/pea-sys/Til/blob/master/PyTroch%E6%9C%AC7_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.5 IMDb（Internet Movie Database）からDataLoaderを作成
本ファイルでは、IMDb（Internet Movie Database）のデータを使用して、感情分析（0：ネガティブ、1：ポジティブ）を2値クラス分類するためのDatasetとDataLoaderを作成します。
※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.5 学習目標
テキスト形式のファイルデータからtsvファイルを作成し、torchtext用のDataLoaderを作成できるようになる
事前準備
書籍の指示に従い、本章で使用するデータを用意します

# 1. IMDbデータセットをtsv形式に変換
Datasetをダウンロードします

※torchtextで標準でIMDbが使える関数があるのですが、今回は今後データセットが用意されていない場合でも対応できるように0から作ります。

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

5万件のデータ（train,testともに2.5万件）です。データidとrating（1-10）でファイル名が決まっています。

rateは10の方が良いです。4以下がnegative、7以上がpositiveにクラス分けされています。

[写経元](https://github.com/YutaroOgawa/pytorch_advanced/blob/master/7_nlp_sentiment_transformer/7-5_IMDb_Dataset_DataLoader.ipynb)

In [0]:
import os
import urllib.request
import zipfile
import tarfile

In [3]:
!git clone https://github.com/YutaroOgawa/pytorch_advanced.git
%cd pytorch_advanced/
%cd 4_pose_estimation

Cloning into 'pytorch_advanced'...
remote: Enumerating objects: 441, done.
remote: Total 441 (delta 0), reused 0 (delta 0), pack-reused 441
Receiving objects: 100% (441/441), 14.62 MiB | 18.04 MiB/s, done.
Resolving deltas: 100% (231/231), done.
/content/pytorch_advanced
/content/pytorch_advanced/4_pose_estimation


In [0]:
# フォルダ「data」が存在しない場合は作成する
data_dir = "data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [0]:
# fastTextの公式の英語学習済みモデル（650MB）をダウンロード。時間が5分ほどかかります
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
save_path = "data/wiki-news-300d-1M.vec.zip"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [0]:
# フォルダ「data」内の「/wiki-news-300d-1M.vec.zip」を解凍する

zip = zipfile.ZipFile("data/wiki-news-300d-1M.vec.zip")
zip.extractall("data/")  # ZIPを解凍
zip.close()  # ZIPファイルをクローズ

# フォルダ「data」内にフォルダ「wiki-news-300d-1M.vec」というものができます。

In [0]:
# IMDbデータセットをダウンロード。30秒ほどでダウンロードできます

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
save_path = "data/aclImdb_v1.tar.gz"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [0]:
# './data/aclImdb_v1.tar.gz'の解凍　1分ほどかかります

# tarファイルを読み込み
tar = tarfile.open('data/aclImdb_v1.tar.gz')
tar.extractall('data/')  # 解凍
tar.close()  # ファイルをクローズ

# フォルダ「data」内にフォルダ「aclImdb」というものができます。

In [0]:
# tsv形式のファイルにします
import glob
import os
import io
import string


# 訓練データのtsvファイルを作成します

f = open('data/IMDb_train.tsv', 'w')

path = 'data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = 'data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

In [0]:
# テストデータの作成

f = open('data/IMDb_test.tsv', 'w')

path = 'data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = 'data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

# 2. 前処理と単語分割の関数を定義

In [9]:
import string
import re

# 以下の記号はスペースに置き換えます（カンマ、ピリオドを除く）。
# punctuationとは日本語で句点という意味です
print("区切り文字：", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 前処理


def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）


def tokenizer_punctuation(text):
    return text.strip().split()


# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 動作を確認します
print(tokenizer_with_preprocessing('I like cats.'))

区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


# DataLoaderの作成

In [0]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
import torchtext


# 文章とラベルの両方に用意します
max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 引数の意味は次の通り
# init_token：全部の文章で、文頭に入れておく単語
# eos_token：全部の文章で、文末に入れておく単語

In [11]:
# フォルダ「data」から各tsvファイルを読み込みます
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 動作確認
print('訓練および検証のデータ数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))

訓練および検証のデータ数 25000
1つ目の訓練および検証のデータ {'Text': ['feroz', 'abbas', 'khan', 's', 'gandhi', 'my', 'father', ',', 'a', 'film', 'that', 'sheds', 'light', 'on', 'the', 'fractured', 'relationship', 'between', 'the', 'mahatma', 'and', 'his', 'son', 'harilal', 'gandhi', '.', 'for', 'a', 'story', 'that', 's', 'as', 'dramatic', 'as', 'the', 'one', 'this', 'film', 'attempts', 'to', 'tell', ',', 'it', 's', 'a', 'pity', 'the', 'director', 'fails', 'to', 'tell', 'it', 'dramatically', '.', 'gandhi', 'my', 'father', 'is', 'narrated', 'to', 'you', 'like', 'that', 'boring', 'history', 'lesson', 'that', 'put', 'you', 'to', 'sleep', 'at', 'school', '.', 'now', 'the', 'film', 'aims', 'to', 'convey', 'one', 'very', 'interesting', 'point', 'the', 'fact', 'that', 'gandhi', 'in', 'his', 'attempt', 'to', 'be', 'a', 'fair', 'person', ',', 'ended', 'up', 'being', 'an', 'unfair', 'father', '.', 'this', 'point', 'is', 'made', 'in', 'the', 'film', 'many', 'times', 'over', ',', 'and', 'one', 'of', 'the', 'examples', 'giv

In [12]:

import random
# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('1つ目の訓練データ', vars(train_ds[0]))

訓練データの数 20000
検証データの数 5000
1つ目の訓練データ {'Text': ['in', 'a', 'movie', 'that', 'follows', 'a', 'struggling', 'actor', ',', 'played', ',', 'evidently', ',', 'by', 'a', 'struggling', 'actor', ',', 'this', 'does', 'no', 'favours', 'for', 'chris', 'klein', '.', 'he', 'struggles', 'to', 'bring', 'anything', 'memorable', 'to', 'the', 'role', 'and', 'meanders', 'on', 'through', 'the', 'shallow', 'script', 'managing', 'to', 'display', ',', 'what', 'could', 'only', 'be', 'described', 'as', ',', 'a', 'bland', 'leading', 'man', '.', 'the', 'story', 'exists', ',', 'but', 'that', 'is', 'all', ',', 'and', 'fails', 'to', 'show', 'any', 'basic', 'start', ',', 'middle', 'and', 'end', 'and', 'the', 'viewer', 'is', 'left', 'shrugging', 'his', 'shoulders', 'feeling', 'as', 'though', 'nothing', 'in', 'the', 'past', 'hour', 'and', 'three', 'quarters', 'has', 'really', 'happened', '.', 'one', 'bright', 'light', 'in', 'the', 'midst', 'of', 'this', 'is', 'fred', 'durst', ',', 'who', 'manages', 'to', 'stand', 'out'

# ボキャブラリーを作成

In [17]:
# torchtextで単語ベクトルとして英語学習済みモデルを読み込みます

from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')


# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", english_fasttext_vectors.dim)
print("単語数：", len(english_fasttext_vectors.itos))

100%|█████████▉| 999358/999994 [01:51<00:00, 8881.52it/s]

1単語を表現する次元数： 300
単語数： 999994


In [18]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 17916個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi

torch.Size([17904, 300])


defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'the': 4,
             '.': 5,
             ',': 6,
             'and': 7,
             'a': 8,
             'of': 9,
             'to': 10,
             'is': 11,
             'it': 12,
             'in': 13,
             'i': 14,
             'this': 15,
             'that': 16,
             's': 17,
             'was': 18,
             'as': 19,
             'for': 20,
             'with': 21,
             'movie': 22,
             'but': 23,
             'film': 24,
             't': 25,
             'on': 26,
             'you': 27,
             'not': 28,
             'he': 29,
             'are': 30,
             'his': 31,
             'have': 32,
             'be': 33,
             'one': 34,
             'all': 35,
             'at': 36,
             'they': 37,
             'by': 38,
             'an': 39,

In [19]:

# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[    2,  5282, 14522,  ...,     4,   244,     3],
        [    2,  1286,    10,  ...,     7,     4,     3],
        [    2,    14,    32,  ...,   587,     7,     3],
        ...,
        [    2,    21,     8,  ...,     5,    12,     3],
        [    2,    34,     9,  ...,     1,     1,     1],
        [    2,   479,   599,  ...,    72,   558,     3]]), tensor([256, 256, 256, 214,  91, 256, 256,  40, 170, 116, 208,  95, 256,  22,
        210, 117, 256, 123, 256, 256,  50, 256,  98, 256]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])


このようにDataLoaderは単語のidを格納しているので、分散表現はディープラーニングモデル側でidに応じて取得してあげる必要があります。

ここまでの内容をフォルダ「utils」のdataloader.pyに別途保存しておき、次節からはこちらから読み込むようにします